Copyright (c) 2022, salesforce.com, inc and MILA.  
All rights reserved.  
SPDX-License-Identifier: BSD-3-Clause  
For full license text, see the LICENSE file in the repo root  
or https://opensource.org/licenses/BSD-3-Clause  

Copyright (c) 2022, salesforce.com, inc and MILA.  
All rights reserved.  
SPDX-License-Identifier: BSD-3-Clause  
For full license text, see the LICENSE file in the repo root  
or https://opensource.org/licenses/BSD-3-Clause  

# This is the notebook to create the datasets and yaml file
Dependency: wbgapi, pandas, numpy

In [ ]:
# pip install wbgapi
import wbgapi as wb
import numpy as np
import pandas as pd
from opt_helper import *
import warnings
warnings.filterwarnings('ignore')

Get convergence population predicted by United Nation

In [ ]:
lasdf = pd.read_csv("UN-pop-pred.csv")
lasdf = lasdf[lasdf["Year"]==2100].reset_index(drop=True)

Get the country classes information from the world bank classification. Class 0 includes countries all over the world. Class 1 to 20 are divided by region X income level.

In [ ]:
cc = pd.read_csv("CountryClass.csv")
countryclass = {i:list(cc[cc["RIG"]==i]["Code"]) for i in range(1,21)}
countryclass[0] = sum([countryclass[i] for i in range(1,21)] ,[])

In [ ]:
# create coalition yamls
# countryclass = extract_group_memberships("coalition.csv")

Get the env protection proportion contribution from the IMF data

In [ ]:
envdf = pd.read_csv("Environmental_Protection_Expenditures_Geo_Avg_Recent_Years_Sum.csv")

Include the tax rate

In [ ]:
taxdf = pd.read_csv("tax_rate.csv")

Include the env protection expenditure for the countries

In [ ]:
import csv
env_pay = {}
with open("Environmental_Protection_Expenditures_Geo_Avg_Recent_Years_Sum.csv", 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        env_pay[row[0]] = row[1]

The series (token) what we are interested.

In [ ]:
# the list of series that we want to query
series_list = ["NY.GDP.MKTP.CD","CM.MKT.LCAP.CD", "SP.POP.TOTL","EN.ATM.CO2E.KT", "NE.CON.TOTL.ZS"]
df = wb.data.DataFrame(series_list, time=range(1960, 2022, 1), labels=True)
df = df.reset_index()

Determine which years are interested and what countries should be excluded (because of lacking in data)

In [ ]:
# retrive country codes stuff and make sure the data are float rather than str
economy_list = list(df["economy"])[:len(set(df["economy"]))]
country_list = list(df["Country"])[:len(set(df["economy"]))]
economy_region_list = []
for x in economy_list:
    if x=="WLD": break
    else: economy_region_list.append(x)

In [ ]:
# add to the exec_code list because no GDP data available
noY=[]
for x in economy_region_list:
    tmp = []
    for y in range(2003, 2021):
        if pd.isnull(get_data_list(df, x, "Y")[1]["YR"+str(y)]):
            tmp.append(0)
        else:
            tmp.append(1)
    if sum(tmp)!=len(range(2003, 2021)):
        noY.append(x)

In [ ]:
exe_code = noY
exc_code = ['YEM','VIR','VEN', 'TKM','SYR', 'MAF', 'SSD', 'SOM', 'SXM', 'SMR', 'MNP', 'NCL', 'NRU', 'LIE', 'XKX', 'PRK', 'IMN', 
            'GRL', 'GIB', 'PYF', 'FRO', 'ERI', 'CUW', 'CHI', 'CYM', 'VGB', 'ABW', 'AND', "TWN"] # exclude codes becasue we don't even have GDP data ever
dict_country = {}
for i in range(len(economy_list)):
    dict_country[economy_list[i]]=country_list[i]
years = list(df.columns)[4:]
df[years] = df[years].apply(pd.to_numeric)

Introducing the carbon intensity

In [ ]:
# include the carbon intensity
for i in range(len(country_list)):
    s = df[df["economy"]==economy_list[i]][df["series"]=="EN.ATM.CO2E.KT"][years].reset_index(drop=True)*1_000_000/df[df["economy"]==economy_list[i]][df["series"]=="NY.GDP.MKTP.CD"][years].reset_index(drop=True)
    s.at[0,"economy"] = economy_list[i]
    s.at[0,"Country"] = country_list[i]
    s.at[0,"series"] = "EN.ATM.CO2E.KD.CD"
    s.at[0, "Series"] = "sigma"
    df = df.append(s)
    df = df.reset_index(drop=True)

In [ ]:
# check those regions who does not have sigma data
count = 0
noco2 = {}
hasco2 = {}
for i in range(len(country_list)):
    if df[df["series"]=="EN.ATM.CO2E.KD.CD"][df["economy"]==economy_list[i]]["YR2018"].isnull().values[0]:
        noco2[economy_list[i]]=country_list[i]
        continue
    else:
        hasco2[economy_list[i]]=country_list[i]
        count+=1

In [ ]:
# get the country list without co2 data
noco2_region_dict = {}
for k,v in noco2.items():
    if k=="WLD": break
    else: noco2_region_dict[k]=v

In [ ]:
# since there is no co2 data for the key countries, 
# we manually find a replacement data for them from other countires
borrowdict = {'PSE': 'EGY',
 'VIR': 'USA',
 'VEN': 'MEX',
 'TCA': 'GBR',
 'MAF': 'FRA',
 'SSD': 'EGY',
 'SXM': 'NLD',
 'SMR': 'ITA',
 'PRI': 'USA',
 'MNP': 'USA',
 'NCL': 'FRA',
 'MCO': 'FRA',
 'MAC': 'CHN',
 'XKX': 'TUR',
 'PRK': "RUS",
 'IMN': 'GBR',
 'HKG': 'CHN',
 'GUM': 'USA',
 'GRL': 'DNK',
 'GIB': 'GBR',
 'PYF': 'FRA',
 'FRO': 'DNK',
 'ERI': 'EGY',
 'CUW': 'NLD',
 'CHI': 'GBR',
 'CYM': 'GBR',
 'VGB': 'GBR',
 'BMU': 'CAN',
 'ABW': 'MEX',
 'ASM': 'USA',}

In [ ]:
# find the country code to index mapping for noco2data available countries
code2idx = {}
for k in noco2_region_dict.keys():
    code2idx[k] = df[df["series"]=="EN.ATM.CO2E.KD.CD"][df["economy"]==k].index.values[0]

In [ ]:
# update the co2 data for noco2 recorded countries by borrowdict
for k in noco2_region_dict.keys(): # borrowdict
    try:
        df.loc[code2idx[k],years] = df[df["series"]=="EN.ATM.CO2E.KD.CD"][df["economy"]==borrowdict[k]][years].iloc[0]
    except:
        print(k)

Fillna for countries without capital data

In [ ]:
# Prepare the (Y,L) data and predict K data by KNN
def get_Y_K_L_pairs(codes, year=None, exc_code=[]):
    if year is None:
        year = "YR2020"
    else:
        year = "YR"+str(year)
    train_data = []
    train_label = []
    train_codes = []
    test_data = []
    test_codes = []
    codes_ = []
    for code in codes:
        if code in exc_code:
            continue
        else:
            label = get_data_list(df, code, "K")[1][year]
            if pd.isnull(label):
                test_data.append([get_data_list(df, code, "Y")[1][year], get_data_list(df, code, "L")[1][year]])
                test_codes.append(code)
            else:
                train_data.append([get_data_list(df, code, "Y")[1][year], get_data_list(df, code, "L")[1][year]])
                train_label.append(label)
                train_codes.append(code)
    return np.array(train_data), np.array(train_label), np.array(test_data), dict(zip(train_codes, train_data)), dict(zip(test_codes, test_data))

In [ ]:
# use KNN to predict those who has Y, L data but don't have K data
for y in range(2003, 2021):
    train_data, train_label, test_data, train_dict, test_dict = get_Y_K_L_pairs(economy_region_list, y, exc_code)
    from sklearn.neighbors import KNeighborsRegressor
    neigh = KNeighborsRegressor(n_neighbors=5)
    neigh.fit(train_data, train_label)
    test_K = neigh.predict(test_data)
    test_region = list(test_dict.keys())
    for i in range(len(test_region)):
        idx = df[df["economy"]==test_region[i]][df["series"]=="CM.MKT.LCAP.CD"]["YR2003"].index.values[0]
        df.loc[idx,"YR"+str(y)]=test_K[i]

Use KNN to fill consumption data， prepare the (Y,L) data and predict C data by KNN

In [ ]:
def get_Y_C_L_pairs(codes, year=None, exc_code=[]):
    if year is None:
        year = "YR2020"
    else:
        year = "YR"+str(year)
    train_data = []
    train_label = []
    train_codes = []
    test_data = []
    test_codes = []
    codes_ = []
    for code in codes:
        if code in exc_code:
            continue
        else:
            label = get_data_list(df, code, "C")[1][year]
            if pd.isnull(label):
                test_data.append([get_data_list(df, code, "Y")[1][year], get_data_list(df, code, "L")[1][year]])
                test_codes.append(code)
            else:
                train_data.append([get_data_list(df, code, "Y")[1][year], get_data_list(df, code, "L")[1][year]])
                train_label.append(label)
                train_codes.append(code)
    return np.array(train_data), np.array(train_label), np.array(test_data), dict(zip(train_codes, train_data)), dict(zip(test_codes, test_data))

In [ ]:
# use KNN to predict those who has Y, L data but don't have K data
for y in range(2003, 2021):
    train_data, train_label, test_data, train_dict, test_dict = get_Y_C_L_pairs(economy_region_list, y, exc_code)
    from sklearn.neighbors import KNeighborsRegressor
    neigh = KNeighborsRegressor(n_neighbors=5)
    neigh.fit(train_data, train_label)
    test_C = neigh.predict(test_data)
    test_region = list(test_dict.keys())
    for i in range(len(test_region)):
        idx = df[df["economy"]==test_region[i]][df["series"]=="NE.CON.TOTL.ZS"]["YR2003"].index.values[0]
        df.loc[idx,"YR"+str(y)]=test_C[i]

In [ ]:
for code in economy_region_list:
    if code in exc_code:
        continue
    else:
        if len(get_data_list(df, code, "C")[0]) == 0:
            print(code)

introduce the tech factor (ATFP) based on Y,K,L

In [ ]:
for i in range(len(economy_list)):
    if economy_list[i] in exc_code: continue
    else:
        s = df[df["economy"]==economy_list[i]][df["series"]=="NY.GDP.MKTP.CD"][years].reset_index(drop=True)/(1000_000_000_000*((df[df["economy"]==economy_list[i]][df["series"]=="CM.MKT.LCAP.CD"][years].reset_index(drop=True)/1000_000_000_000)**0.3*(df[df["economy"]==economy_list[i]][df["series"]=="SP.POP.TOTL"][years].reset_index(drop=True)/1000_000_000)**0.7))
        s.at[0,"economy"] = economy_list[i]
        s.at[0,"Country"] = country_list[i]
        s.at[0,"series"] = "ATFP"
        s.at[0, "Series"] = "A"
        df = df.append(s)
        df = df.reset_index(drop=True)

Gather the results of the time series accross different regions

In [ ]:
raw_results = {}
for x in economy_region_list:
    if x in exc_code:
        continue
    else:
        raw_results[x]={}
        raw_results[x]["TS_Y"] = get_data_list(df, x, "Y")
        raw_results[x]["TS_A"] = get_data_list(df, x, "A")
        raw_results[x]["TS_K"] = get_data_list(df, x, "K")
        raw_results[x]["TS_L"] = get_data_list(df, x, "L")
        raw_results[x]["TS_sigma"] = get_data_list(df, x, "sigma")
        raw_results[x]["TS_C"] = get_data_list(df, x, "C")
        raw_results[x]["La"] = list(lasdf[lasdf["Code"]==x]["Population (future projections)"])[0]
        raw_results[x]["mitigation"] = get_env_data_list(envdf, x)
        raw_results[x]["saving"] = 1 - int(raw_results[x]["TS_C"][0][-1])/100
        try:
            raw_results[x]["tax"] = get_tax_data_list(taxdf, x)
        except:
            raw_results[x]["tax"] = 0

Merge regions from data from small regions

Align the time series length of A, K and L

In [ ]:
def packup_regions(raw_data, groupnum):
    output = {}
    starts = []
    ends = []
    for token in ["TS_A", "TS_K", "TS_L"]: 
        for c in countryclass[groupnum]:
            if c in exc_code:
                continue
            starts.append(raw_data[c][token][3])
            ends.append(raw_data[c][token][4])
    s = max(starts)
    e = min(ends)
    
    for token in ["TS_A", "TS_K", "TS_L"]: 
        output[token] = []
        for c in countryclass[groupnum]:
            if c in exc_code:
                continue
            for i in range(s,e+1):
                if i==s:
                    output[token].append([])
                output[token][-1].append(raw_data[c][token][1]["YR"+str(i)])
    output["La_s"] = [raw_results[c]["La"] for c in countryclass[groupnum] if c not in exc_code]
    output["sigmai_s"] = [raw_results[c]["TS_sigma"][0][-1] for c in countryclass[groupnum] if c not in exc_code]
    output["tax_s"] = [raw_results[c]["tax"] for c in countryclass[groupnum] if c not in exc_code]
    output["mitigation_s"] = [raw_results[c]["mitigation"] for c in countryclass[groupnum] if c not in exc_code]
    output["saving_s"] = [raw_results[c]["saving"] for c in countryclass[groupnum] if c not in exc_code]
    return output

An output template

In [ ]:
default = {"_RICE_CONSTANT":
  {"xgamma": 0.3, # in CAP Eq 5 the capital elasticty

  # A rice data
  "xA_0": 0,
  "xg_A": 0,
  "xdelta_A": 0.0214976314392836,
  # L
  "xL_0": 1397.715000, # in POP population at the staring point
  "xL_a": 1297.666000, # in POP the expected population at convergence
  "xl_g": 0.04047275402855734, # in POP control the rate to converge
  # K
  "xK_0": 93.338152,
  "xa_1": 0,
  "xa_2": 0.00236 ,
  "xa_3": 2,

  # xsigma_0: 0.5201338309755572
  "xsigma_0": 0.215}}

Get the parameters of the dynamics from the timeseries gather the above

In [ ]:

para_result = {i:{} for i in range(1,21)}
for i in range(1,21):
    print(countryclass[i])
    a = merge_region_dict(packup_regions(raw_results, i))
    para_result[i]["xl_g"] = get_pop_lg(a["Ls"], a["Las"])
    para_result[i]["xL_a"] = a["Las"]/1_000_000
    para_result[i]["xL_0"] = a["Ls"][-1]/1_000_000
    para_result[i]["xg_A"],para_result[i]["xdelta_A"] = get_gA_deltaA(a["As"])
    para_result[i]["xA_0"] = a["As"][-1]
    para_result[i]["xK_0"] = a["Ks"][-1]/1_000_000_000_000
    para_result[i]["xsigma_0"] = a["sigmas"]/(1-0.05)
    para_result[i]["xtax"] = a["taxs"]
    para_result[i]["xmitigation_0"] = a["mitigations"]
    para_result[i]["xsaving_0"] = a["savings"]

In [ ]:

para_result = {k:{} for k in countryclass.keys()}
for i in countryclass.keys():
    print(countryclass[i])
    a = merge_region_dict(packup_regions(raw_results, i))
    para_result[i]["xl_g"] = get_pop_lg(a["Ls"], a["Las"])
    para_result[i]["xL_a"] = a["Las"]/1_000_000
    para_result[i]["xL_0"] = a["Ls"][-1]/1_000_000
    para_result[i]["xg_A"],para_result[i]["xdelta_A"] = get_gA_deltaA(a["As"])
    para_result[i]["xA_0"] = a["As"][-1]
    para_result[i]["xK_0"] = a["Ks"][-1]/1_000_000_000_000
    para_result[i]["xsigma_0"] = a["sigmas"]/(1-0.05)
    para_result[i]["xtax"] = a["taxs"]
    para_result[i]["xmitigation_0"] = a["mitigations"]
    para_result[i]["xsaving_0"] = a["savings"]

Add the "rest of the world" region and used the worldwide parameters for it

In [ ]:
# Calculate the number of population which is not covered
popworldwide = list(df[df["economy"]=="WLD"][df["series"]=="SP.POP.TOTL"]["YR2020"])[0]
popcovvered = sum([raw_results[x]["TS_L"][0][-1] for x in economy_region_list if x not in exc_code])
popnotcovered = popworldwide - popcovvered
# calculate the estimate worldwide convergence population
covered_convergence_pop = sum([raw_results[x]["La"] for x in economy_region_list if x not in exc_code])
not_covvered_convergence_pop = popnotcovered*covered_convergence_pop/popcovvered

In [ ]:
# check the worldwide properties
i=0
wld = {}
a = merge_region_dict(packup_regions(raw_results, i))
wld["xl_g"] = get_pop_lg(a["Ls"], a["Las"])
wld["xL_a"] = a["Las"]/1_000_000
wld["xL_0"] = a["Ls"][-1]/1_000_000
wld["xg_A"],wld["xdelta_A"] = get_gA_deltaA(a["As"])
wld["xA_0"] = a["As"][-1]
wld["xK_0"] = a["Ks"][-1]/1_000_000_000_000
wld["xsigma_0"] = a["sigmas"]/(1-0.05)
wld["xtax"] = a["taxs"]
wld["xmitigation_0"] = a["mitigations"]
wld["xsaving_0"] = a["savings"]

In [ ]:
# get the rest of the world params
rest_region = wld.copy()
rest_region["xL_a"] = not_covvered_convergence_pop/1_000_000
rest_region["xL_0"] = popnotcovered/1_000_000
rest_region["xK_0"] = popnotcovered*wld["xK_0"]/popcovvered
para_result[21] = rest_region

We split USA, CHN and IND because they are too large economics.

In [ ]:
def split_region(datadict, code, start, end, splits=[], gamma=0.3):
    A, K, L =[],[],[]
    for i in range(start, end+1):
        A.append(datadict[code]["TS_A"][1]["YR"+str(i)])
        K.append(datadict[code]["TS_K"][1]["YR"+str(i)])
        L.append(datadict[code]["TS_L"][1]["YR"+str(i)])
    A,K,L = np.array(A),np.array(K),np.array(L)
    La = datadict[code]["La"]
    Y = A*K**gamma*L**(1-gamma)
    splits = np.array(splits)
    if len(splits)==0:
        splits = np.random.rand(4)
    if sum(splits)!=1:
        splits = splits/sum(splits)
    LEN = len(splits)
    Ys = Y.reshape(1,-1)
    Ys = np.repeat(Ys,LEN,axis=0)
    for i in range(len(splits)):
        Ys[i] = Ys[i]*splits[i]
    Ls = L.reshape(1,-1)
    Ls = np.repeat(Ls,LEN,axis=0)
    for i in range(len(splits)):
        Ls[i] = Ls[i]*splits[i]
    multiplier = np.clip(np.exp(np.random.normal(0.5,0.5,LEN)), 0.75, 2)
    As = A.reshape(1,-1)
    As = np.repeat(As,LEN,axis=0)
    for i in range(len(multiplier)):
        As[i] = As[i]*multiplier[i]
    Ks = (Ys/(As*Ls**(1-gamma)))**(1/gamma)
    Las = La*splits
    return As, Ks, Ls, Las

In [ ]:
splitted_data = {}
splitted_data["USA"] = split_region(raw_results, "USA", 1975, 2020, [0.5, 0.33, 0.17])
splitted_data["CHN"] = split_region(raw_results, "CHN", 2003, 2020, [0.5, 0.33, 0.17])
splitted_data["IND"] = split_region(raw_results, "IND", 2000, 2020, [0.5, 0.33, 0.17])

In [ ]:
para_result.update({i:{} for i in range(22,31)})
for i in range(0,3):
    para_result[i+22]["xl_g"] = get_pop_lg(splitted_data["USA"][2][i], splitted_data["USA"][2][i])
    para_result[i+22]["xL_a"] = splitted_data["USA"][3][i]/1_000_000
    para_result[i+22]["xL_0"] = splitted_data["USA"][2][i][-1]/1_000_000
    para_result[i+22]["xg_A"],para_result[i+22]["xdelta_A"] = get_gA_deltaA(splitted_data["USA"][0][i])
    para_result[i+22]["xA_0"] = splitted_data["USA"][0][i][-1]
    para_result[i+22]["xK_0"] = splitted_data["USA"][1][i][-1]/1_000_000_000_000
    para_result[i+22]["xsigma_0"] = raw_results["USA"]["TS_sigma"][0][-1]/0.95
    para_result[i+22]["xtax"] = para_result[1]["xtax"] # USA country code 1
    para_result[i+22]["xmitigation_0"] = para_result[1]["xmitigation_0"]
    para_result[i+22]["xsaving_0"] = para_result[1]["xsaving_0"]
for i in range(0,3):
    para_result[i+25]["xl_g"] = get_pop_lg(splitted_data["CHN"][2][i], splitted_data["CHN"][2][i])
    para_result[i+25]["xL_a"] = splitted_data["CHN"][3][i]/1_000_000
    para_result[i+25]["xL_0"] = splitted_data["CHN"][2][i][-1]/1_000_000
    para_result[i+25]["xg_A"],para_result[i+25]["xdelta_A"] = get_gA_deltaA(splitted_data["CHN"][0][i])
    para_result[i+25]["xA_0"] = splitted_data["CHN"][0][i][-1]
    para_result[i+25]["xK_0"] = splitted_data["CHN"][1][i][-1]/1_000_000_000_000
    para_result[i+25]["xsigma_0"] = raw_results["CHN"]["TS_sigma"][0][-1]/0.95
    para_result[i+25]["xtax"] = para_result[8]["xtax"] # CHN country code 8
    para_result[i+25]["xmitigation_0"] = para_result[8]["xmitigation_0"]
    para_result[i+25]["xsaving_0"] = para_result[8]["xsaving_0"]
for i in range(0,3):
    para_result[i+28]["xl_g"] = get_pop_lg(splitted_data["IND"][2][i], splitted_data["IND"][2][i])
    para_result[i+28]["xL_a"] = splitted_data["IND"][3][i]/1_000_000
    para_result[i+28]["xL_0"] = splitted_data["IND"][2][i][-1]/1_000_000
    para_result[i+28]["xg_A"],para_result[i+28]["xdelta_A"] = get_gA_deltaA(splitted_data["IND"][0][i])
    para_result[i+28]["xA_0"] = splitted_data["IND"][0][i][-1]
    para_result[i+28]["xK_0"] = splitted_data["IND"][1][i][-1]/1_000_000_000_000
    para_result[i+28]["xsigma_0"] = raw_results["IND"]["TS_sigma"][0][-1]/0.95
    para_result[i+28]["xtax"] = para_result[10]["xtax"]
    para_result[i+28]["xmitigation_0"] = para_result[10]["xmitigation_0"]
    para_result[i+28]["xsaving_0"] = para_result[10]["xsaving_0"]

In [ ]:
para_result = {k:{} for k in raw_results.keys()}
for i in raw_results.keys():
    try:
        a = raw_results[i]
    except:
        print(i)
        continue
    para_result[i]["xl_g"] = get_pop_lg(np.array(a["TS_L"][0]), np.array(a["La"]))
    para_result[i]["xL_a"] = np.array(a["La"])/1_000_000
    para_result[i]["xL_0"] = np.array(a["TS_L"][0])[-1]/1_000_000
    para_result[i]["xg_A"], para_result[i]["xdelta_A"] = get_gA_deltaA(np.array(a["TS_A"][0]))
    para_result[i]["xA_0"] = np.array(a["TS_A"][0])[-1]
    para_result[i]["xK_0"] = np.array(a["TS_K"][0])[-1]/1_000_000_000_000
    para_result[i]["xsigma_0"] = np.array(a["TS_sigma"][0][-1])/(1-0.05)
    para_result[i]["xtax"] = np.array(a["tax"])
    para_result[i]["xmitigation_0"] = np.array(a["mitigation"])
    para_result[i]["xsaving_0"] = np.array(a["saving"])
rest_region = wld.copy()
rest_region["xL_a"] = not_covvered_convergence_pop/1_000_000
rest_region["xL_0"] = popnotcovered/1_000_000
rest_region["xK_0"] = popnotcovered*wld["xK_0"]/popcovvered
para_result["REST"] = rest_region

Write Yaml files for the merged regions

In [ ]:
write_yaml_files(para_result, "C:\\Users\\tiany\\Documents\\mila-sfdc-climate-econ\\region_yamls")

Backup the dataframe used in this notebook

In [ ]:
df.to_csv("backup_wbdf_2023_11_15.csv")